# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.core.workspace import Workspace
from azureml.core.compute import ComputeTarget
from azureml.core.compute.amlcompute import AmlCompute
from azureml.exceptions import ComputeTargetException
from azureml.core.experiment import Experiment
from azureml.core.model import Model

from azureml.core import Environment
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
from azureml.core.webservice import Webservice

from azureml.train.automl.automlconfig import AutoMLConfig
from azureml.widgets.run_details import RunDetails

from azureml.automl.core.shared import constants

from pprint import pprint
import pandas as pd

import logging
import joblib

import json
import requests

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.

Dat:[ Breast Cancer Prediction Dataset](https://www.kaggle.com/merishnasuwal/breast-cancer-prediction-dataset)

This machine learning program detects the presence (or absence) of breast cancer from pertinent data regarding physical characteristics.
An understanding of the data can be had at https://www.kaggle.com/merishnasuwal/breast-cancer-prediction-dataset/discussion/66975#509394


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

The dataset is external. It is manually downloaded as a csv and then uploaded to a publicly acccessible github account:
'https://github.com/dntrply/nd00333-capstone/blob/master/dataset/Breast_cancer_data.csv'

In [ ]:
ds = TabularDatasetFactory.from_delimited_files('https://github.com/dntrply/nd00333-capstone/raw/master/dataset/Breast_cancer_data.csv')
df = ds.to_pandas_dataframe()
df

,mean_radius,mean_texture,mean_perimeter,mean_area,mean_smoothness,diagnosis
0,17.99,10.38,122.80,1001.0,0.11840,0
1,20.57,17.77,132.90,1326.0,0.08474,0
2,19.69,21.25,130.00,1203.0,0.10960,0
3,11.42,20.38,77.58,386.1,0.14250,0
4,20.29,14.34,135.10,1297.0,0.10030,0
...,...,...,...,...,...,...
564,21.56,22.39,142.00,1479.0,0.11100,0
565,20.13,28.25,131.20,1261.0,0.09780,0
566,16.60,28.08,108.30,858.1,0.08455,0
567,20.60,29.33,140.10,1265.0,0.11780,0


In [ ]:
df.describe()

,mean_radius,mean_texture,mean_perimeter,mean_area,mean_smoothness,diagnosis
count,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000
mean,14.127292,19.289649,91.969033,654.889104,0.096360,0.627417
std,3.524049,4.301036,24.298981,351.914129,0.014064,0.483918
min,6.981000,9.710000,43.790000,143.500000,0.052630,0.000000
25%,11.700000,16.170000,75.170000,420.300000,0.086370,0.000000
50%,13.370000,18.840000,86.240000,551.100000,0.095870,1.000000
75%,15.780000,21.800000,104.100000,782.700000,0.105300,1.000000
max,28.110000,39.280000,188.500000,2501.000000,0.163400,1.000000


In [ ]:
# Split the dtaaset so that a small fraction may be used for prediction
train_ds, predict_ds = ds.random_split(percentage=0.99, seed=44)

In [ ]:
train_df = train_ds.to_pandas_dataframe()
train_df

,mean_radius,mean_texture,mean_perimeter,mean_area,mean_smoothness,diagnosis
0,17.99,10.38,122.80,1001.0,0.11840,0
1,20.57,17.77,132.90,1326.0,0.08474,0
2,19.69,21.25,130.00,1203.0,0.10960,0
3,11.42,20.38,77.58,386.1,0.14250,0
4,20.29,14.34,135.10,1297.0,0.10030,0
...,...,...,...,...,...,...
560,21.56,22.39,142.00,1479.0,0.11100,0
561,20.13,28.25,131.20,1261.0,0.09780,0
562,16.60,28.08,108.30,858.1,0.08455,0
563,20.60,29.33,140.10,1265.0,0.11780,0


In [ ]:
ws = Workspace.from_config()

# choose a name for experiment
experiment=Experiment(ws, 'experiment-capstone-automl')  # Experiment name in Azure ML

In [ ]:
# Next, let's use if it exists, or create if required, a compute cluster to be used by the ML

# Access the compute cluster. If it exists, we will have the compute object. 
# If it does not exist, an exception will be thrown upon which the compute cluster is created
try:
    cc = ComputeTarget(workspace=ws, name='COMPUTE-AUTOML')
except ComputeTargetException:
    # Failed to obtain the compute cluster object
    # In all likelihood, a compute cluster of that name has not been created
    # Attempt to create the compute cluster
    # First set up the configuration

    # Specify the configuration of the compute cluster
    cc_cfg = AmlCompute.provisioning_configuration(vm_size='Standard_DS3_v2', min_nodes=1, max_nodes=4)
    cc = ComputeTarget.create(workspace=ws, name='COMPUTE-AUTOML', provisioning_configuration=cc_cfg)

# At this point - we have access to the compute cluster object. Wait for the compute target to complete provisioing
cc.wait_for_completion(show_output='True')

Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

This project is a classification issue. More so, it is a binary classification issue as teh outcome is whether the wine is of a good quality or not.

AUC_weighted is an apporpriate metric to target for a binary classification.
[Set up AutoML training with Python](https://docs.microsoft.com/en-us/azure/machine-learning/how-to-configure-auto-train)

It is generally recommended to enable early stopping as it is possible that after a while no further improvement in the model is feasible.

There is enrally limited to no benefit to using a large number of cross validations. In this instance, we have set it to 3.

In [ ]:
# TODO: Put your automl settings here

automl_settings = {
    "iterations" : 5,
    "experiment_timeout_minutes" : 30,
    "enable_early_stopping" : True,
    "iteration_timeout_minutes" : 5,
    "max_concurrent_iterations" : 5,
    "max_cores_per_iteration" : -1,
    "n_cross_validations" : 3,
    "primary_metric" : 'AUC_weighted',
    "verbosity" : logging.INFO,
}

# Provide the remainder of the settings/configuration
# Note that we are not providing a validation data set
# 


# TODO: Put your automl config here
automl_config = AutoMLConfig(
    compute_target = cc,
    task='classification',
    training_data=train_ds,
    label_column_name='diagnosis',
    featurization='auto',
    model_explainability=True,
    debug_log='capstone_automl.log',
    **automl_settings)

In [ ]:
# TODO: Submit your experiment
automl_run = experiment.submit(automl_config)

Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
experiment-capstone-automl,AutoML_044bd13f-c225-4a30-8081-95a15a34fdf5,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [ ]:
RunDetails(automl_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

Current provisioning state of AmlCompute is "Deleting"



In [ ]:
automl_run.wait_for_completion(show_output=True)

Experiment,Id,Type,Status,Details Page,Docs Page
experiment-capstone-automl,AutoML_044bd13f-c225-4a30-8081-95a15a34fdf5,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

****************************************************************************************************

TYPE:         High cardinality feature detection
STATUS

{'runId': 'AutoML_044bd13f-c225-4a30-8081-95a15a34fdf5',
 'target': 'COMPUTE-AUTOML',
 'status': 'Completed',
 'startTimeUtc': '2021-10-31T08:20:25.924102Z',
 'endTimeUtc': '2021-10-31T08:24:44.740131Z',
 'services': {},
 'properties': {'num_iterations': '5',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'AUC_weighted',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '3',
  'target': 'COMPUTE-AUTOML',
  'AMLSettingsJsonString': '{"path":null,"name":"experiment-capstone-automl","subscription_id":"81cefad3-d2c9-4f77-a466-99a7f541c7bb","resource_group":"aml-quickstarts-162443","workspace_name":"quick-starts-ws-162443","region":"southcentralus","compute_target":"COMPUTE-AUTOML","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"enable_batch_run":false,"enable_run_restructure":false,"start_auxiliary_runs_before_parent_complete":false,"enable_code_generation":false,"iterati

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [ ]:
def print_model(model, prefix=""):
    for step in model.steps:
        print(prefix + step[0])
        if hasattr(step[1], 'estimators') and hasattr(step[1], 'weights'):
            pprint({'estimators': list(e[0] for e in step[1].estimators), 'weights': step[1].weights})
            print()
            for estimator in step[1].estimators:
                print_model(estimator[1], estimator[0]+ ' - ')
        elif hasattr(step[1], '_base_learners') and hasattr(step[1], '_meta_learner'):
            print("\nMeta Learner")
            pprint(step[1]._meta_learner)
            print()
            for estimator in step[1]._base_learners:
                print_model(estimator[1], estimator[0]+ ' - ')
        else:
            pprint(step[1].get_params())
            print()

In [ ]:
automl_best_run, automl_best_model = automl_run.get_output()

automl_best_run_metrics = automl_best_run.get_metrics()

print(f'********** Best AutoML accuracy: {automl_best_run_metrics.get("accuracy")}')
print(f'********** printing Best AutoML run:\n{automl_best_run}\n\nPrinting model:')

print_model(automl_best_model)

Package:azureml-automl-runtime, training version:1.35.0, current version:1.34.0
Package:azureml-core, training version:1.35.0, current version:1.34.0
Package:azureml-dataprep, training version:2.23.2, current version:2.22.2
Package:azureml-dataprep-rslex, training version:1.21.2, current version:1.20.1
Package:azureml-dataset-runtime, training version:1.35.0, current version:1.34.0
Package:azureml-defaults, training version:1.35.0, current version:1.34.0
Package:azureml-interpret, training version:1.35.0, current version:1.34.0
Package:azureml-mlflow, training version:1.35.0, current version:1.34.0
Package:azureml-pipeline-core, training version:1.35.0, current version:1.34.0
Package:azureml-responsibleai, training version:1.35.0, current version:1.34.0
Package:azureml-telemetry, training version:1.35.0, current version:1.34.0
Package:azureml-train-automl-client, training version:1.35.0, current version:1.34.0
Package:azureml-train-automl-runtime, training version:1.35.0, current versi

********** Best AutoML accuracy: 0.9150155728169912
********** printing Best AutoML run:
Run(Experiment: experiment-capstone-automl,
Id: AutoML_044bd13f-c225-4a30-8081-95a15a34fdf5_1,
Type: azureml.scriptrun,
Status: Completed)

Printing model:
datatransformer
{'enable_dnn': False,
 'enable_feature_sweeping': True,
 'feature_sweeping_config': {},
 'feature_sweeping_timeout': 86400,
 'featurization_config': None,
 'force_text_dnn': False,
 'is_cross_validation': True,
 'is_onnx_compatible': False,
 'observer': None,
 'task': 'classification',
 'working_dir': '/mnt/batch/tasks/shared/LS_root/mounts/clusters/nb-compute/code/Users/odl_user_162443'}

MaxAbsScaler
{'copy': True}

XGBoostClassifier
{'base_score': 0.5,
 'booster': 'gbtree',
 'colsample_bylevel': 1,
 'colsample_bynode': 1,
 'colsample_bytree': 1,
 'gamma': 0,
 'learning_rate': 0.1,
 'max_delta_step': 0,
 'max_depth': 3,
 'min_child_weight': 1,
 'missing': nan,
 'n_estimators': 100,
 'n_jobs': -1,
 'nthread': None,
 'objective':

In [ ]:
print(automl_run.get_metrics())

{'experiment_status': ['DatasetEvaluation', 'FeaturesGeneration', 'DatasetFeaturization', 'DatasetFeaturizationCompleted', 'DatasetCrossValidationSplit', 'ModelSelection'], 'experiment_status_description': ['Gathering dataset statistics.', 'Generating features for the dataset.', 'Beginning to fit featurizers and featurize the dataset.', 'Completed fit featurizers and featurizing the dataset.', 'Generating individually featurized CV splits.', 'Beginning model selection.'], 'matthews_correlation': 0.8173399858673968, 'accuracy': 0.9150155728169912, 'f1_score_macro': 0.9079528521962598, 'balanced_accuracy': 0.9029506281201982, 'f1_score_weighted': 0.9144111660082285, 'AUC_macro': 0.9746663638677168, 'log_loss': 0.2179164154744797, 'weighted_accuracy': 0.9255239750352541, 'recall_score_macro': 0.9029506281201982, 'recall_score_micro': 0.9150155728169912, 'precision_score_macro': 0.9144736632975768, 'AUC_micro': 0.976294008295778, 'average_precision_score_weighted': 0.9765616567620965, 'pre

In [ ]:
# Create the outputs directory
if 'outputs' not in os.listdir():
    os.mkdir('outputs')

In [ ]:
#TODO: Save the best model
joblib.dump(automl_best_model, os.path.join('outputs','best_automl.pkl'))

['outputs/best_automl.pkl']

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [ ]:
# Refer - https://docs.microsoft.com/en-us/azure/machine-learning/how-to-deploy-and-where?tabs=python

# Tutorial: Deploy an image classification model in Azure Container Instances -
# https://docs.microsoft.com/en-us/azure/machine-learning/tutorial-deploy-models-with-aml

# Register the model
# registered_model = automl_run.register_model(model_name='wine-taste-automl', description=c_constants.DEPLOYED_AUTOML_MODEL_DESCRIPTION)
registered_model = automl_best_run.register_model(model_path=constants.MODEL_PATH, 
                                                model_name='breast-cancer-automl', 
                                                description='Breast Cancer detection using Azure AutoML',
                                                tags={'Method of execution':'AutoML'},
                                                properties={'Accuracy':automl_best_run_metrics['accuracy']})
print(f'{automl_run.model_id}')
print(f'{registered_model.name}  {registered_model.id}  {registered_model.version}')


None
breast-cancer-automl  breast-cancer-automl:2  2


In [ ]:
# download the scoring file and the environmrnt file

automl_best_run.download_file(constants.SCORING_FILE_PATH, os.path.join('outputs', 'scoring.py'))
automl_best_run.download_file(constants.CONDA_ENV_FILE_PATH, os.path.join('outputs', 'best_run_environment.yml'))

In [ ]:
# Create an inference config

inference_config = InferenceConfig(
    environment=Environment.from_conda_specification(name='myenv', file_path=os.path.join('outputs', 'best_run_environment.yml')),
    source_directory='outputs',
    entry_script='scoring.py',
)

aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)


In [ ]:

service = Model.deploy(workspace=ws,
                       name='breast-cancer-service',
                       models=[registered_model],
                       inference_config=inference_config,
                       deployment_config=aci_config,
                       overwrite=True)
service.wait_for_deployment(show_output=True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2021-10-31 08:25:15+00:00 Creating Container Registry if not exists.
2021-10-31 08:25:15+00:00 Registering the environment.
2021-10-31 08:25:16+00:00 Use the existing image.
2021-10-31 08:25:16+00:00 Generating deployment configuration.
2021-10-31 08:25:17+00:00 Submitting deployment to compute.
2021-10-31 08:25:20+00:00 Checking the status of deployment breast-cancer-service..
2021-10-31 08:31:07+00:00 Checking the status of inference endpoint breast-cancer-service.
Succeeded
ACI service creation operation finished, operation "Succeeded"


In [ ]:
logs = service.get_logs()

for line in logs.split('\n'):
    print(line)


2021-10-31T08:30:15,449078400+00:00 - iot-server/run 
2021-10-31T08:30:15,455034900+00:00 - rsyslog/run 
2021-10-31T08:30:15,454315200+00:00 - nginx/run 
2021-10-31T08:30:15,455739600+00:00 - gunicorn/run 
Dynamic Python package installation is disabled.
Starting HTTP server
rsyslogd: /azureml-envs/azureml_50e1173ccd55f57fca3555fc5c69dc71/lib/libuuid.so.1: no version information available (required by rsyslogd)
EdgeHubConnectionString and IOTEDGE_IOTHUBHOSTNAME are not set. Exiting...
2021-10-31T08:30:15,951314400+00:00 - iot-server/finish 1 0
2021-10-31T08:30:15,952731100+00:00 - Exit code 1 is normal. Not restarting iot-server.
Starting gunicorn 20.1.0
Listening at: http://127.0.0.1:31311 (73)
Using worker: sync
worker timeout is set to 300
Booting worker with pid: 98
SPARK_HOME not set. Skipping PySpark Initialization.
Generating new fontManager, this may take some time...
Initializing logger
2021-10-31 08:30:19,036 | root | INFO | Starting up app insights client
logging socket was 

TODO: In the cell below, send a request to the web service you deployed to test it.

In [ ]:
# To enable ApplicationInsights on the service (webservice), 
# * first access the endpoint using the name assigned at the time of deployment
# * next update webservice parameters such as enabling application insights (enable_app_insights)

webservice = Webservice(
    workspace = ws,
    name='deploy-studio'
)

webservice.update(
    enable_app_insights=True
)

# At this point application insights (logging is enabled) and can be
# checked in the GUI in AutoML studio

In [ ]:
# URL for the web service, should be similar to:
# 'http://8530a665-66f3-49c8-a953-b82a2d312917.eastus.azurecontainer.io/score'

# From the tail end of the code at
# https://docs.microsoft.com/en-us/azure/machine-learning/how-to-deploy-and-where?tabs=python
# - Deploy machine learning models to Azure

scoring_uri = webservice.scoring_uri

# If the service is authenticated, set the key or token
# key, _ = webservice.get_keys()

# Set the appropriate headers
headers = {"Content-Type": "application/json"}
# headers["Authorization"] = f"Bearer {key}"


In [ ]:
predict_data = predict_ds.to_pandas_dataframe()
predict_label = predict_data.pop('diagnosis')

In [ ]:
# Convert to JSON string
score_data = json.dumps({'data': predict_data.to_dict(orient='records')})

score_data

'{"data": [{"mean_radius": 20.18, "mean_texture": 23.97, "mean_perimeter": 143.7, "mean_area": 1245.0, "mean_smoothness": 0.1286}, {"mean_radius": 10.49, "mean_texture": 19.29, "mean_perimeter": 67.41, "mean_area": 336.1, "mean_smoothness": 0.09989}, {"mean_radius": 9.777, "mean_texture": 16.99, "mean_perimeter": 62.5, "mean_area": 290.2, "mean_smoothness": 0.1037}, {"mean_radius": 12.62, "mean_texture": 23.97, "mean_perimeter": 81.35, "mean_area": 496.4, "mean_smoothness": 0.07903}]}'

In [ ]:
# Set the content type
headers = {'Content-Type': 'application/json'}
# If authentication is enabled, set the authorization header
# headers['Authorization'] = f'Bearer {key}'

# Make the request and display the predictions
resp = requests.post(scoring_uri, score_data, headers=headers)
print(f'{resp.json()}')

# Print the actual diagnosis
print(f'{predict_label}')

{"result": [0, 1, 1, 1]}
0    0
1    1
2    1
3    1
Name: diagnosis, dtype: int64


TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
logs = webservice.get_logs()

for line in logs.split('\n'):
    print(line)



2021-10-31T07:50:20,058634900+00:00 - rsyslog/run 
2021-10-31T07:50:20,060108900+00:00 - iot-server/run 
2021-10-31T07:50:20,069655800+00:00 - gunicorn/run 
Dynamic Python package installation is disabled.
Starting HTTP server
2021-10-31T07:50:20,094637000+00:00 - nginx/run 
rsyslogd: /azureml-envs/azureml_5006f59e0e58c3c9418bb0a068e4845e/lib/libuuid.so.1: no version information available (required by rsyslogd)
EdgeHubConnectionString and IOTEDGE_IOTHUBHOSTNAME are not set. Exiting...
2021-10-31T07:50:20,343290100+00:00 - iot-server/finish 1 0
2021-10-31T07:50:20,346207800+00:00 - Exit code 1 is normal. Not restarting iot-server.
Starting gunicorn 20.1.0
Listening at: http://127.0.0.1:31311 (71)
Using worker: sync
worker timeout is set to 300
Booting worker with pid: 99
SPARK_HOME not set. Skipping PySpark Initialization.
Generating new fontManager, this may take some time...
Initializing logger
2021-10-31 07:50:22,410 | root | INFO | Starting up app insights client
logging socket was 

In [ ]:
# Clean up any resources
# Delete the Webservice
# delete the compute cluster

webservice.delete()
cc.delete()

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.
